# Visualizing Transit Reliaiblity With APC Data

In this workbook we will explore a few different ways to create clean and compelling visualizations of transit reliability by making heavy use of the `altair` visualization module. If you are not familiar with Altair, take a look at Altair's [simple charts guide](https://altair-viz.github.io/altair-tutorial/notebooks/02-Simple-Charts.html), and Jim Vallandingham's [introduction to altair](https://vallandingham.me/altair_intro.html).

In this workbook, we will be using data that has been cleaned and processed using the techniques introduced in lecture and in the workbook entitled "Exploring Reliability with APC Data". Specifically, we are going to make use of two files:
1. A cleaned and simplified dataset of all `delta` values on Calgary Transit's Route 3 Southbound (`ROUTEHND = 195`), between January 6 and January 27, 2019 with the following columns:
    * `detail_id` is the unique `DETAILHND` value from the dataset
    * `trip_id` is the trip the bus is serving
    * `start_time` is the start time of the recorded event
    * `end_time` is the end time of the recorded event
    * `start_delta` is the schedule deivation in minutes at the start of the event (negative is early)
    * `end_delta` is the schedule deviation in minutes at the end of the event (negative is early)
    * `ons` are the number of people detected as boarding at that stop
    * `offs` are the number of people detected as alighting at that stop
    * `theta` is the number of passengers estimated on board (using a balancing system done by the APC device)
    * `stop_seq` is the ordinal stop number from the start of the route (0 is the departure terminus)
    * `stopway` is the name of the stop
    * `stop_long` is the longitude of the stop (in a currently unknown projection system)
    * `stop_lat` is the latitude of the stop (in a currently unknown projection system)
2. A processed and aggregated dataset of daily on-time performance (OTP) based on a threshold range from 2 minutes early to 5 minutes late, with the following columns:
    * `date` is the date of measurement (includes data from midnight to midnight of that day)
    * `route_name` is the number of the route (includes both directions)
    * `on_time` is the number of observed on-time arrivals (in the interval [-2, 5])
    * `all_counts` is the total number of observed arrivals
    * `otp` is the on-time performance, in percent. It is calculated as `100 * (on_time/all_counts)`

The goal of this workbook is twofold: First, to get comfortable with some of the different types of charts and visualizations possible, and how to construct them with Altair, and second how to take the base visualization provded and elevate it by following some of the visualization principles of clarity, data/ink ratios, and showing the data.

Let's start as we usually do by loading the relevant modules and specifying our data location. You will want to download an extract "Lab 3 - Calgary APC Data Cleaned" from Quercus, unzip, and place the `calgary_apc_clean` files in the `data' folder specifically.

In [311]:
import pandas as pd
import altair as alt
import numpy as np
import os
import datetime
data_folder = '../data/calgary_apc_clean'

In [312]:
deltas = pd.read_csv(os.path.join(data_folder, 'calgary_apc_clean_2019-01-06_2019-01-27_rhnd_195.csv'))
otp = pd.read_csv(os.path.join(data_folder, 'otp_all_routes_2019-01-06_2019-01-27.csv'))
# Pandas doesn't automagically know our datetime columns are datetime colums
# So we specify them ourselves.
deltas['start_time'] = pd.to_datetime(deltas['start_time'])
deltas['end_time'] = pd.to_datetime(deltas['end_time'])

# As a basic look let's describe our data
display(deltas.describe())
display(otp.describe())

,detail_id,trip_id,start_delta,end_delta,ons,offs,theta,stop_seq,stop_long,stop_lat
count,31401.000000,31401.000000,31401.000000,31401.000000,31401.000000,31401.000000,31401.000000,31401.000000,31401.000000,3.140100e+04
mean,727.686348,15379.709977,1.096528,1.617712,1.665743,1.657272,14.424923,33.912519,-4995.120983,5.658442e+06
std,429.033010,4113.790212,3.360219,3.164257,2.785481,3.303393,11.127603,20.723826,579.874759,5.182100e+03
min,11.000000,7672.000000,-9.933333,-8.316667,0.000000,0.000000,-40.000000,0.000000,-6262.000000,5.649197e+06
25%,396.000000,17144.000000,-0.616667,-0.250000,0.000000,0.000000,6.000000,16.000000,-5468.000000,5.655092e+06
50%,677.000000,17230.000000,0.666667,1.000000,1.000000,1.000000,12.000000,32.000000,-5014.000000,5.658526e+06
75%,1007.000000,17293.000000,2.533333,2.900000,2.000000,2.000000,20.000000,46.000000,-4391.000000,5.663483e+06
max,3199.000000,25046.000000,19.783333,51.650000,78.000000,84.000000,81.000000,74.000000,-4379.000000,5.666473e+06


,route_name,on_time,all_counts,otp
count,3828.000000,3828.000000,3828.000000,3828.000000
mean,271.554598,339.778997,411.401254,79.888006
std,296.935140,433.183655,529.153633,17.038207
min,1.000000,1.000000,1.000000,6.250000
25%,55.000000,20.000000,28.000000,75.485572
50%,115.000000,206.000000,251.000000,84.197100
75%,439.250000,484.000000,582.250000,89.720388
max,896.000000,3980.000000,4862.000000,100.000000


## Visualizing Schedule Deviations

Let's start by looking at our raw data of arrival and departure times and schedule deviations. Let's start by looking at various ways to "show all the data" for a given period of interest. For our purposes, let's use the end of the event (the departure of the bus) as our representative time. This means we'll use the `end_time` column.

### Trajectories
One nice way to see all the data and maintain the temporal nature of the data is by plotting trajectories of vehicles. This can be done in various ways, including following schedule deviation over stops, or by plotting Marey diagrams. We will take a look at both.

As an example, let's plot trajectories for a given operating day in various ways. Let's start by isolating out a single operating day by selecting an appropriate start and end timestamp to filter our `end_time` column on. We'll also drop data we consider to be "outliers"

In [313]:
study_start = datetime.datetime(2019, 1, 16, 3, 0, 0)
study_end   = datetime.datetime(2019, 1, 17, 2, 59, 59)

study_deltas = deltas[(deltas['end_time'] >= study_start) & (deltas['end_time'] <= study_end)].copy()
study_deltas.head()

,detail_id,trip_id,start_time,end_time,start_delta,end_delta,ons,offs,theta,stop_seq,stop_id,stopway,stop_long,stop_lat
385,36,17207,2019-01-16 05:19:01,2019-01-16 05:19:36,-0.983333,-0.400000,5,0,5,0,N5203,NB @ Sandstone Terminal,-6262,5666473
386,185,17209,2019-01-16 07:29:20,2019-01-16 07:31:46,-2.666667,-0.233333,9,0,9,0,N5203,NB @ Sandstone Terminal,-6262,5666473
387,383,17211,2019-01-16 09:48:53,2019-01-16 09:51:51,-1.116667,1.850000,3,0,3,0,N5203,NB @ Sandstone Terminal,-6262,5666473
388,582,17213,2019-01-16 12:05:24,2019-01-16 12:10:46,-4.600000,0.766667,9,0,9,0,N5203,NB @ Sandstone Terminal,-6262,5666473
389,800,17215,2019-01-16 14:22:42,2019-01-16 14:31:18,-7.300000,1.300000,5,0,5,0,N5203,NB @ Sandstone Terminal,-6262,5666473


#### Marey Diagrams
One way to show example trajectories is through Marey diagrams (often called stringline or time-distance diagrams). In this case we'll show both the trajectory of the vehicle as well as it's scheduled trajectory. To do this we will first need to calcualte the scheduled time based on our delta measurements.

In [314]:
study_deltas['sched_start'] = study_deltas['start_time'] - pd.to_timedelta(study_deltas['start_delta'], unit='m')
study_deltas['sched_end'] = study_deltas['end_time'] - pd.to_timedelta(study_deltas['end_delta'], unit='m')
study_deltas.head()

,detail_id,trip_id,start_time,end_time,start_delta,end_delta,ons,offs,theta,stop_seq,stop_id,stopway,stop_long,stop_lat,sched_start,sched_end
385,36,17207,2019-01-16 05:19:01,2019-01-16 05:19:36,-0.983333,-0.400000,5,0,5,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 05:19:59.999999980,2019-01-16 05:20:00.000000000
386,185,17209,2019-01-16 07:29:20,2019-01-16 07:31:46,-2.666667,-0.233333,9,0,9,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 07:32:00.000000020,2019-01-16 07:31:59.999999980
387,383,17211,2019-01-16 09:48:53,2019-01-16 09:51:51,-1.116667,1.850000,3,0,3,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 09:50:00.000000020,2019-01-16 09:50:00.000000000
388,582,17213,2019-01-16 12:05:24,2019-01-16 12:10:46,-4.600000,0.766667,9,0,9,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 12:10:00.000000000,2019-01-16 12:09:59.999999980
389,800,17215,2019-01-16 14:22:42,2019-01-16 14:31:18,-7.300000,1.300000,5,0,5,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 14:30:00.000000000,2019-01-16 14:30:00.000000000


Now we have the relevant time data for us to plot, but we need to adjust the dataset's form a little bit to make it easier to encode the information the way Altair likes it. 

Also, to make our visualization a bit more sane, let's look only keep trips which ran during the AM peak period (7am to 9am).

In [315]:
marey = study_deltas[['trip_id', 'end_time', 'sched_end', 'stop_seq']].copy()
# Let's find the trips that ran during the period
# If you changed the study date above, you'll need to change it here also.
trip_ids = marey[ (marey.end_time >= "2019-01-16 07:00")  & (marey.end_time < "2019-01-16 09:00")].trip_id
marey = marey[marey.trip_id.isin(trip_ids)]
# Let's name our columns, with pretty names for the ones we are going to 'melt' in a second here
marey.columns = ['trip_id', 'Actual', 'Scheduled', 'stop_seq']
# The melt function lets us create a long-form table to encode better
marey = marey.melt(id_vars=['trip_id', 'stop_seq'])
# And now we plot
alt.Chart(marey).mark_line().encode(
    alt.X('value:T'),
    alt.Y('stop_seq:Q'),
    alt.Color('variable:N'),
    alt.Detail('trip_id:N')
)

alt.Chart(...)

This is a good start, but we can definitely clean things up a little bit. Let's make the following adjustments:
* Better axis names
* Changed channels to have trips colored by `trip_id`, but stroke style by actual/scheduled times
* Addition of a title and description
* Removal of the background grid
* Change the dimensions of the plot
* Change the size of the lettering
* Change fonts

In [316]:
alt.Chart(marey).mark_line().encode(
    alt.X('value:T', title="Time on January 16, 2019"),
    alt.Y('stop_seq:Q', title='Stop Sequence'),
    alt.Color('trip_id:N', legend=None),
    alt.StrokeDash('variable:N', title=""),
).properties(
    title={
      "text": ["Morning Peak Marey Diagrams for Route 3 Southbound"], 
      "subtitle": ["Actual and scheduled trajectories are shown for the morning (7-9am) peak period on January 16, 2019."]
    },
    width=600, 
    height=300
).configure(
    font='Lato'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    titleFontSize=14
).configure_title(
    fontSize=20,
    anchor='start',
    subtitleFontSize=12,
    subtitlePadding=5
).configure_legend(
    labelFontSize=12,
    orient='top-left'
)

alt.Chart(...)

#### Schedule Deviation Trajectory
It might also be useful to look at the schedule deviation trajectory for all the buses on a given day. This can help us see anomalies and possible data issues, but it also provides a nice way to talk about the on-time performance metric as well as see general patterns or trends we might not otherwise see. Let's start by getting the data we need and getting it out onto a plot.

In [317]:
traj = study_deltas[['trip_id', 'end_time', 'end_delta', 'stop_seq']].copy()
alt.Chart(traj).mark_line().encode(
    alt.X('stop_seq:Q'),
    alt.Y('end_delta:Q'),
    alt.Detail('trip_id')
)

alt.Chart(...)

Pretty messy in this case, but we can perhaps add some additional insight by showing the data we want a bit clearer and in the context of other performance measures:
* Thinned out lines for clearer picture
* Crop our Y axis (but not our data) to max out at 15 minutes
* Add an `area` in the background which shows where on-time performance falls within this context.
* Add a `punctuality` metric in the foreground showing the average deviation at each stop

In this example we'll do our first *layered* plot. With simple plots like this (i.e. no facets or small multiples) we can simply create two `Chart` objects and add them together before applying our desired styles.

In [335]:
# We can create a dataset of puntuality quickly using Panda's groupby function
punctuality = traj[['end_delta', 'stop_seq']].groupby('stop_seq', as_index=False).mean()

base = alt.Chart(traj).mark_line(size=1, color='#3F3F3F', opacity=0.4, clip=True).encode(
    alt.X('stop_seq:Q', title='Stop Sequence'),
    alt.Y('end_delta:Q', title='Schedule Deviation (min)', scale=alt.Scale(domain=([-7, 15]))),
    alt.Detail('trip_id'),
    # alt.Color('hours(end_time):O')
)

punct = alt.Chart(punctuality).mark_line(color='#2464B0').encode(
    alt.X('stop_seq:Q'),
    alt.Y('end_delta:Q')
)

area = alt.Chart(pd.DataFrame({
    'x': [0, traj.stop_seq.max()],
    'y': [-2, -2],
    'y2': [5, 5]
})).mark_area(fill='#EEEEEE').encode(
    alt.X('x:Q'),
    alt.Y('y:Q'),
    alt.Y2('y2:Q')
)

(area + base + punct).properties(
    title={
      "text": ["Schedule Deviation of Route 3 Southbound Buses on January 16, 2019"], 
      "subtitle": ["Each individual southbound bus run is plotted as a trajectory, showing its deviation from the scheduled arrival time", 
                   "at each stop. A value of 0 indiciates an on-time bus; a negative value indicates an early bus. The grey box indicates",
                   "an 'on-time' definition between 2 minutes early and 5 minutes late. The blue line shows the average schedule deviation",
                   "at that particular stop, also known as 'punctuality'."],
    },
    width=600, 
    height=300
).configure(
    font='Lato'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    titleFontSize=14
).configure_title(
    fontSize=20,
    anchor='start',
    subtitleFontSize=12,
    subtitlePadding=5
).configure_legend(
    labelFontSize=12,
    orient='top-left'
)


alt.LayerChart(...)

### Distribution Plots

Another way we can communicate the randomness in the system while keeping some granularity and spatiotemporal insight is to provide the viewer a sense of the distribution of our deltas accross stops throughout the day, or on a broader scale accross days throughout the study period.

Let's start with a simple look at how deltas are distributed accross stops throughout our single-day `study_delta` dataframe. We can start by using the `mark_bloxplot()` function built into Altair:

In [319]:
alt.Chart(study_deltas).mark_boxplot().encode(
    alt.X('stop_seq:O'),
    alt.Y('end_delta:Q')
)

alt.Chart(...)

We can certainly learn about our dataset this way, the default boxplot setting isn't particularly clean or pretty - and the bars that are shown aren't super intuitive for our purpose, extending from the first/third quartile by 1.5 times that interquartile range.

Let's draw directly from the suggestions provided by Edward Tufte in *Visual Display of Quantitative Information* and construct a "dot and whiskers" plot instead, where:
* The central moment will be the median `delta` value
* Bars will extend from the 75th percentile to the 95th percentile and from the 5th percentile to the 25th percentile

To do this, we need to build our dot plots and whiskers plots separately, and layer them on top of each other. To do *that*, we need to first determine the desired percentiles for our data. Pandas (version `0.25.0` and above) lets you used named aggregation for this. We will also keep track of our outliers so we can add them to the plot.

In [320]:
# A nice custom way of getting the percentiles we want
dot_whisker = study_deltas[['stop_seq', 'end_delta']].groupby('stop_seq', as_index=False).agg(
    p05 = ('end_delta',lambda x: x.quantile(0.05)),
    p25 = ('end_delta',lambda x: x.quantile(0.25)), 
    p50 = ('end_delta',lambda x: x.quantile(0.50)),
    p75 = ('end_delta',lambda x: x.quantile(0.75)),
    p95 = ('end_delta',lambda x: x.quantile(0.95)),
)
display(dot_whisker.head())
outliers = []
for idx, dw in dot_whisker.iterrows():
    outliers.append(study_deltas[(study_deltas.stop_seq == dw.stop_seq) & ((study_deltas.end_delta < dw.p05) | (study_deltas.end_delta > dw.p95))])

outliers = pd.concat(outliers, axis='index')
outliers.head()

,stop_seq,p05,p25,p50,p75,p95
0,0,-2.591667,-0.183333,0.433333,1.583333,12.016667
1,1,-0.186667,0.316667,0.733333,1.316667,10.793333
2,2,-0.065000,0.187500,0.533333,1.391667,3.355000
3,3,-0.963333,0.000000,0.400000,0.866667,2.080000
4,4,-1.140833,-0.370833,0.058333,0.683333,2.570833


,detail_id,trip_id,start_time,end_time,start_delta,end_delta,ons,offs,theta,stop_seq,stop_id,stopway,stop_long,stop_lat,sched_start,sched_end
390,1247,17219,2019-01-16 19:32:05,2019-01-16 19:35:29,12.083333,15.483333,1,0,1,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 19:20:00.000000020,2019-01-16 19:20:00.000000020
404,1278,17250,2019-01-16 18:09:55,2019-01-16 18:13:04,11.916667,15.066667,1,0,1,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 17:57:59.999999980,2019-01-16 17:57:59.999999980
408,1013,17189,2019-01-16 11:47:03,2019-01-16 11:47:12,-2.950000,-2.800000,2,0,2,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 11:50:00.000000000,2019-01-16 11:50:00.000000000
409,1194,17191,2019-01-16 14:01:58,2019-01-16 14:06:44,-8.033333,-3.266667,4,0,4,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 14:09:59.999999980,2019-01-16 14:10:00.000000020
419,994,17338,2019-01-16 17:53:36,2019-01-16 17:54:45,15.600000,16.750000,2,0,2,0,N5203,NB @ Sandstone Terminal,-6262,5666473,2019-01-16 17:38:00.000000000,2019-01-16 17:38:00.000000000


Now we can assemble the plot. Let's carry over the idea of putting the OTP box on the plot to provide people some understanding of how that metric comapres to the distribution.

In [321]:
upper_whisker = alt.Chart(dot_whisker).mark_bar(width=1, color='#708090').encode(
    alt.X("stop_seq:O", title=''),
    alt.Y("p75:Q"),
    alt.Y2("p95:Q")
)

lower_whisker = alt.Chart(dot_whisker).mark_bar(width=1, color='#708090').encode(
    alt.X("stop_seq:O", title=''),
    alt.Y("p05:Q"),
    alt.Y2("p25:Q")
)

median = alt.Chart(dot_whisker).mark_circle(size=20, fill='#F58426').encode(
    alt.X("stop_seq:O", title='Stop Sequence'),
    alt.Y("p50:Q", title='Schedule Deviation (min)')
)

outlier = alt.Chart(outliers).mark_circle(size=8, fill='#708090').encode(
    alt.X("stop_seq:O", title='Stop Sequence'),
    alt.Y("end_delta:Q", title='Schedule Deviation (min)')
)

area = alt.Chart(pd.DataFrame({
    'x': [0, traj.stop_seq.max()],
    'y': [-2, -2],
    'y2': [5, 5]
})).mark_area(fill='#EEEEEE', opacity=0.7).encode(
    alt.X('x:O'),
    alt.Y('y:Q'),
    alt.Y2('y2:Q')
)

# The order we add them is the order they are drawn (leftmost is on the bottom)
(area + upper_whisker + lower_whisker + median + outlier).properties(
    title={
      "text": ["Distribution of Schedule Deviations on Calgary Transit Route 3 Southbound on Januray 16, 2019"], 
      "subtitle": ["For each stop on the route, the distribution of differences between the actual and scheduled arrival times at stops are shown using a dot-and-whiskers plot. The orange dot indicates the", 
                   "median (half of all the deviations are less than this value), while the blue lines span between the 5th and 25th percentile and the 75th and 95th percentile of the data. The grey box", 
                   "denotes a common measure of on-time performance which counts buses arriving between 1 minute early and 5 minutes late as 'on time'."],
    },
    width=1200, 
    height=600
).configure(
    font='Lato'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    titleFontSize=14
).configure_title(
    fontSize=20,
    anchor='start',
    subtitleFontSize=12,
    subtitlePadding=5
)

alt.LayerChart(...)

#### Small Multiple Density Plots
Instead of using various 'moments' (median, percentiles) to show the distribution, we can also provide a more direct visualization of the distribuiton by showing the actual density of the plots. This gives people a chance to notice things like bimodal distribuitons or other shape-specific characteristics.

We could show an actual histogram with bars, but we are going to do a *density transform* using a common method called a [Kernel Density Estmation](https://en.wikipedia.org/wiki/Kernel_density_estimation). We will also use facets to create small multiples with Altair - this can be done using the `alt.Row()` or `alt.Column()` features, or by using the facet command. Faceting can get tricky in Altair (and makes layering more complex!), you may have to do some googling to get what you need out of it.

In [322]:
alt.Chart(study_deltas[study_deltas.stop_seq < 15]).transform_density(
    'end_delta',
    as_ = ['end_delta', 'density'],
    groupby = ['stop_seq']
).mark_line(size=1.5).encode(
    alt.X('end_delta:Q', title="Schedule Deviation (min)"),
    alt.Y('density:Q', title=None),
    alt.Row('stop_seq:O', title='Stop Sequence')
).properties(
    title={
      "text": ["Distribution of Schedule Deviations"], 
      "subtitle": [
        "Kernel Density Estimate for Route 3 Southbound on January 16, 2019.",
        "Early buses are shown with negative deviations. First 15 stops shown.",
      ]
    },
    width=400, 
    height=25
).configure(
    font='Lato'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    titleFontSize=12,
).configure_axisY(
    domain=False,
    labels=False,
    ticks=False
).configure_title(
    fontSize=20,
    anchor='start',
    subtitleFontSize=12,
    subtitlePadding=5
).configure_facet(
    spacing=0
).configure_header(
    titleFontSize=12
)

alt.Chart(...)

#### Buffer Time Index
As discussed in class, a buffer time index uses two parts of a distribution (typically the 95th percentile and the median or mean) to calculate a delay index. We are going to calculate this for each link between stops based on our study data.

We are therefore going to need to calculate travel times between stops. We are dealing with data which only records at a stop, so we are guaranteed that if we sort the data by `trip_id` and `stop_seq` that we will get a perfect set of stops.

There is more than one way to solve this problem: We could interpolate between the stops using some estimation of speed or something similar, but it may also be that we have enough data that we can get a representative set of pairs of stops, at least enough to calculate a buffer time index.

In [323]:
# Let's keep only the columns we need, and sort them by trip and sequence so they're in order
link_times = study_deltas[['trip_id', 'end_time', 'stop_seq']].sort_values(by=['trip_id', 'stop_seq']).copy()
link_times = pd.concat([link_times.iloc[:-1], link_times.shift(-1).iloc[:-1]], axis='columns')
# Rename our columns for sanity
link_times.columns = ['f_trip_id', 'f_end_time', 'f_seq', 't_trip_id', 't_end_time', 't_seq']
# Turn our ID columns back into integers
link_times['t_seq'] = link_times['t_seq'].astype(int)
link_times['t_trip_id'] = link_times['t_trip_id'].astype(int)
link_times = link_times[(link_times.f_seq == link_times.t_seq - 1) & (link_times.f_trip_id == link_times.t_trip_id)]

# Now we can calculate a travel time!
link_times['travel_time'] = (link_times['t_end_time'] - link_times['f_end_time']).dt.total_seconds()
link_times.head()


,f_trip_id,f_end_time,f_seq,t_trip_id,t_end_time,t_seq,travel_time
393,17144,2019-01-16 08:03:16,0,17144,2019-01-16 08:05:00,1,104.0
29739,17144,2019-01-16 08:06:05,3,17144,2019-01-16 08:07:00,4,55.0
1795,17144,2019-01-16 08:09:30,7,17144,2019-01-16 08:10:14,8,44.0
3835,17144,2019-01-16 08:14:52,12,17144,2019-01-16 08:15:29,13,37.0
16431,17144,2019-01-16 08:15:29,13,17144,2019-01-16 08:16:30,14,61.0


Now we have our stop-to-stop trip times, lets do a BTI calculation

In [324]:
bti = link_times[['f_seq', 'travel_time']].groupby('f_seq', as_index=False).agg(
    p50 = ('travel_time',lambda x: x.quantile(0.50)),
    p95 = ('travel_time',lambda x: x.quantile(0.95)),
    count = ('travel_time', 'count')
)
bti['bti'] = (bti.p95 - bti.p50)/bti.p50
bti.describe()

,f_seq,p50,p95,count,bti
count,70.000000,70.000000,70.000000,70.000000,70.000000
mean,35.671429,61.328571,90.588571,15.257143,0.460654
std,21.737379,35.220515,65.002960,11.464736,0.504097
min,0.000000,32.000000,32.900000,1.000000,0.000000
25%,17.250000,42.000000,57.250000,4.250000,0.123088
50%,34.500000,51.000000,74.775000,12.500000,0.389314
75%,52.750000,60.875000,101.825000,25.000000,0.560123
max,73.000000,207.000000,427.050000,40.000000,3.028774


Here we can see we run into a bit of a data volume issue - for the calculation and filtering we did, there's not quite enough data to get a solid coverage of the BTI calculation. For now, let's keep data with a count greater than 10 and make a nice plot with it.

Instead of a line plot (which might mislead the viewer into suggesting that there are connecting phenomenon when there might not be) or a standard bar chart, let's create a "lollipop" chart, which keeps the essentials of a bar chart while removing some of the key data.

For one more bit of clarity, let's relabel the stop sequence to run from 1 upwards. People in general understand that better.

In [325]:
bti_plot = bti[bti['count'] >= 10].copy()
bti_plot['seq'] = bti_plot['f_seq'] + 1
bars = alt.Chart(bti_plot).mark_bar(color="#B8B8BD", width=2).encode(
    alt.X('seq:Q', title="", scale=alt.Scale(domain=[1, 73], nice=False), axis=alt.Axis(tickCount=bti_plot.seq.max())),
    alt.Y('bti:Q', title="Buffer Time Index")
)

points = alt.Chart(bti_plot).mark_circle(color="#123A6D", size=60, opacity=1.0).encode(
    alt.X('seq:Q', title="Stop Sequence"),
    alt.Y('bti:Q', title="")
)

text = alt.Chart(bti_plot).mark_text(align='center', baseline='bottom', dy=-8, size=7).encode(
    alt.X("seq:Q"),
    alt.Y('bti:Q', title=""),
    alt.Text('bti', format=',.2f')
)
(bars+points+text).properties(
    title={
      "text": ["Buffer Time Indices for Link Travel Times, Route 3 Southbound, January 6 - 27, 2019"], 
      "subtitle": ["Each stop on the route with sufficient data "],
    },
    width=1024, 
    height=200
).configure(
    font='Lato'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    titleFontSize=14
).configure_title(
    fontSize=20,
    anchor='start',
    subtitleFontSize=12,
    subtitlePadding=5
)

alt.LayerChart(...)

## Visualizing Metrics

Now let's take a look at the metrics we calculated for on-time performance, and look at some of the possible ways we can visualize that data.

First, we need to make a call about how much data we want for a given OTP calculation to be comfortable with what it indicates. One observation is probably not enough, 50 is probably plenty. Let's use a threshold of 20 counts as our filter.

Now we need to think about what we want to show in this case. Let's start by looking at some stats for our cleaned-up dataset:

In [326]:
otp_clean = otp[otp.all_counts >= 20]
otp_clean.describe()

,route_name,on_time,all_counts,otp
count,3012.000000,3012.000000,3012.000000,3012.000000
mean,160.215803,429.940571,520.343625,81.234137
std,190.883565,447.602417,547.890717,12.631885
min,1.000000,2.000000,20.000000,6.666667
25%,43.000000,131.000000,162.000000,77.370766
50%,91.000000,305.000000,372.000000,84.145052
75%,156.000000,576.000000,684.250000,88.931138
max,896.000000,3980.000000,4862.000000,100.000000


### Grid of Values
Seems like the OTP ranges from about 6% all the way up to 100%, but we'll catch most of the data if we look at 70% OTP or above on a scale.

In [327]:
alt.Chart(otp_clean).mark_square(size=50).encode(
    alt.Y('monthdate(date):T', title='Date'),
    alt.X('route_name:N', title='Route Number'),
    alt.Color('otp:Q', title="OTP (%)", scale=alt.Scale(domain=[70, 100], scheme='orangered')), 
    # alt.Size('all_counts:Q', title="Counts (%)")
).properties(
    title={
        "text": "On-Time Performance and Data Coverage for All Routes",
        "subtitle": "Between January 7 and 26, 2019. Scaled for 70 to 100 percent."
    },
    width=1800,
    height=250
).configure(
    font='Lato'
).configure_axis(
        grid=False, 
        titleFontSize=18, 
        labelFontSize=8
).configure_axisX(
    labelPadding=5
).configure_view(
    strokeWidth=0
).configure_title(
    subtitleFontSize = 20, 
    fontSize=26, 
    anchor='start'
).configure_legend(titleFontSize=18, labelFontSize=12)

alt.Chart(...)

### Small Multiples

We could also look at this from a small multiples approach. Can we create a nice OTP over time sparkline for each route (let's pick a subset) and see if we can compare?

In [328]:
alt.Chart(otp_clean[otp_clean.route_name < 30]).mark_line(size=1.5).encode(
    alt.Y('otp:Q', title=None, scale=alt.Scale(domain=[50, 100])),
    alt.X('monthdate(date):T', title=None),
    alt.Row('route_name:N', title=None), 
    # alt.Size('all_counts:Q', title="Counts (%)")
).properties(
    title={
        "text": "On-Time Performance On Routes 1-29, January 6-27, 2019",
        "subtitle": "Each line's bar indicates a range from 50% to 100%. On-time buses range from 2 minutes early to 5 minutes late."
    },
    width=800,
    height=15
).configure(
    font='Lato'
).configure_view(
    strokeWidth=0
).configure_axis(
    grid=False,
    titleFontSize=12,
).configure_axisY(
    labels=False,
    ticks=False
).configure_title(
    fontSize=20,
    anchor='start',
    subtitleFontSize=12,
    subtitlePadding=5
).configure_facet(
    spacing=0,
).configure_header(
    titleFontSize=12
)

alt.Chart(...)

## Exercises
Try out the exercises below to explore how you might tweak or drastically change the visualizations.

1. Pick one of the visualizations and change some aethetic parts of it (font, color, stroke or shape, domains). Find a style you like that is clear and expressive but different from the ones in here.
2. The deltas dataset contains information about boarding and alighting passengers. Instead of just calculating the schedule deviation, instead come up with a passenger-weighted schedule deviation. Try out two different methods of calculating this calculation and visualise both methods. Which one do you like better?
3. (Advanced) using Altair only (i.e. no post-editing), add an area box to the small multiples KDE plot showing where on-time performance would be considered. Hint: You will need to completely change how plots are faceted in order to layer them - happy Googling!)